In [ ]:
from StyleGAN2 import Generator

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
from PIL import Image, ImageFile
from pickle import load, dump
import cv2
import time
import argparse
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def showImage(image):
    %matplotlib inline
    import matplotlib.pyplot as plt

    PIL = transforms.ToPILImage()
    ToTensor = transforms.ToTensor()

    img = PIL(image)
    fig = plt.figure(dpi=200)
    ax = fig.add_subplot(1, 1, 1) # (row, col, num)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(img)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
num_mapping = 8
image_size = 64
max_depth = int(np.log2(image_size)) - 2
def num_fmap(stage):
    base_size = image_size * 2
    fmap_base = base_size * 4
    fmap_max = base_size // 2
    fmap_decay = 1.0
    return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
feed_dim = num_fmap(0)
net = Generator(max_depth, num_fmap, num_mapping).to(device)
net.load_state_dict(torch.load('weight_G.pth', map_location=device))

In [ ]:
style_feeds = [torch.randn(1, feed_dim).to(device)]
image, style = net(style_feeds)
showImage(image[0])

In [ ]:
feeds1 = style_feeds

In [ ]:
style_feeds = [torch.randn(1, feed_dim).to(device)]
image, style = net(style_feeds)
showImage(image[0])

In [ ]:
feeds2 = style_feeds

In [ ]:
%matplotlib widget
fig = plt.figure(dpi=200)
ax = fig.add_subplot(1, 1, 1) # (row, col, num)
ax.set_xticks([])
ax.set_yticks([])
images = []
for i in range(300):
    l = i / 300.0
    x = l * feeds1[0] + (1 - l) * feeds2[0]
    image, _ = net([x])
    PIL = transforms.ToPILImage()
    img = PIL(image[0])
    images += [[plt.imshow(img, animated=True)]]
ani = animation.ArtistAnimation(fig, images, interval=10, repeat_delay=1000)
ani.save('anim.gif', writer="imagemagick")
plt.show()